In [1]:
%reload_ext autoreload
%autoreload 2
!pip install -r requirements.txt
!playwright install

In [8]:
from lib.llm import (get_text_and_image_completions, parse_markdown_output, create_prompt_from_template)
from lib.scraper import get_html_and_screenshot
import nest_asyncio
import asyncio
import time
import os


async def generate_variations(original_url, preview_url, project_dir, selector):

    os.makedirs(f"{project_dir}/generated", exist_ok=True)

    print('Getting the HTML and screenshot of the original page...')
    original_html, original_screenshot = await get_html_and_screenshot(original_url, selector)
    
    for iteration in range(0, 6):

        print(f'Running iteration {iteration}...')

        print('Running the assessment...')
        goals = [
            "Improve conversion rate",
            "Make the styles more consistent with the brand",
            "Improve the overall look and feel of the form"
        ]
        assessment_prompt = create_prompt_from_template(
            "prompts/assessment.txt", 
            goals=goals,
            html=original_html.strip()
        )
        instructions = get_text_and_image_completions(assessment_prompt, image_data=original_screenshot)
        print(instructions)

        print('Generating CSS variation...')
        generation_prompt = create_prompt_from_template(
            "prompts/generation.txt",
            instructions=instructions,
            html=original_html.strip()
        )
        raw_output = get_text_and_image_completions(generation_prompt, image_data=original_screenshot)
        parsed_output = parse_markdown_output(raw_output)
        with open(project_dir + f'generatedStyles.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

        print("Sleeping for 3 second...")
        time.sleep(3)

        print('Getting the HTML and screenshot of the preview page...')
        generated_html, generated_screenshot = await get_html_and_screenshot(preview_url + '?generatedStyles', selector)

        print('Assessing the quality of the generated styles...')
        verification_prompt = create_prompt_from_template(
            "prompts/verification.txt",
            html=generated_html.strip()
        )
        adjustments = get_text_and_image_completions(verification_prompt, image_data=generated_screenshot) 
        print(adjustments)

        print('Applying adjustments...')
        adjustment_prompt = create_prompt_from_template(
            "prompts/adjustment.txt",
            adjustments=adjustments,
            html=generated_html.strip()
        )
        raw_output = get_text_and_image_completions(adjustment_prompt, image_data=generated_screenshot)
        parsed_output = parse_markdown_output(raw_output)

        with open(project_dir + f'generatedStyles.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

        with open(project_dir + f'generated/styles{iteration}.css', 'w') as text_file:
            text_file.write('\n'.join(parsed_output['css']))

original_url = "https://main--wknd--hlxsites.hlx.live/"
preview_url = "http://localhost:3000/"
selector = '.featured-article.block'
project_dir = '/Users/tsaplin/Work/Sources/mystique-wknd/'

nest_asyncio.apply()
asyncio.get_event_loop().run_until_complete(generate_variations(original_url, preview_url, project_dir, selector))



Getting the HTML and screenshot of the original page...
Running iteration 0...
Running the assessment...
- Remove unnecessary inline styles.
- Simplify the background color for the entire form.
- Adjust font size of the article to ensure readability.
- Use a consistent font family across the article text.
- Centralize text alignment for better visual hierarchy.
- Increase line height for better text readability.
- Use a consistent color scheme for headers and body text.
- Ensure button text transformation is consistent with branding.
- Change the button color to a more brand-consistent color.
- Add padding to the button for better spacing.
- Utilize CSS classes for styling instead of inline styles.
- Apply responsive design techniques for various screen sizes.
Generating CSS variation...
Sleeping for 3 second...
Getting the HTML and screenshot of the preview page...
Assessing the quality of the generated styles...
- Remove redundant inline styles that match the default values (e.g., `a